In [146]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from tqdm import tqdm
from src.cdr_bench.io_utils.io import read_features_hdf5_dataframe
from src.cdr_bench.features.physchem import calculate_descriptors
from src.cdr_bench.scoring.scoring import tanimoto_int_similarity_matrix_numba, euclidean_distance_square_numba
from src.cdr_bench.scoring.network_stat import generate_networks_for_thresholds, calculate_network_metrics
from src.cdr_bench.scoring.scaffold_stat import calculate_scaffold_frequencies_and_f50
from src.cdr_bench.scoring.chemsim_stat import calculate_similarity_statistics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Calculate diversity in the datasets and perform scaffold analysis

## Set up a path to the datasets

In [118]:
path_2_datasets = Path('../datasets/')

## Set up some threshold values used for calculations

In [ ]:
similarity_thresholds = [0.7]#[0.5, 0.6, 0.7, 0.8, 0.9]

## Iterate over a directory to get the results

In [ ]:
network_analysis_ls = []
chemdiv_analysis_ls = []

In [ ]:
for dataset in tqdm(path_2_datasets.glob('*.h5')):
    # Read and preprocess the data
    df_features = read_features_hdf5_dataframe(dataset)
    df_features.reset_index(inplace=True)
    df_features.rename(columns={'index': 'cid'}, inplace=True)
    dataset_name = dataset.stem

    # Prepare molecule and scaffold data
    df_features['mol'] = df_features['smi'].parallel_apply(lambda x: Chem.MolFromSmiles(x))
    df_features['scaffold'] = df_features['mol'].parallel_apply(lambda x: Chem.MolToSmiles(MurckoScaffold.GetScaffoldForMol(x)))

    # Calculate physicochemical descriptors and diversity
    physchem_descriptors = np.vstack(df_features['mol'].parallel_apply(lambda x: calculate_descriptors(x)).values)
    intra_dataset_distances = euclidean_distance_square_numba(physchem_descriptors)
    physchem_sim = calculate_similarity_statistics(intra_dataset_distances)
    physchem_sim_prefixed = {f"{key}_physchem": value for key, value in physchem_sim.items()}

    # Calculate scaffold frequencies and F50
    f50 = calculate_scaffold_frequencies_and_f50(df_features['scaffold'].tolist())

    # Initialize chemdiv_df with dataset name and F50 metric
    chemdiv_df = pd.DataFrame({'Dataset': [dataset_name], 'F50': [f50]})
    chemdiv_df = chemdiv_df.assign(**physchem_sim_prefixed)

    # Process Morgan and MACCS descriptors
    for desc in ['mfp_r2_1024', 'maccs_keys']:
        fingerprints = np.vstack(df_features[desc].tolist()).astype(np.float64)

        # Calculate similarity matrix and statistics
        sim_mat = tanimoto_int_similarity_matrix_numba(fingerprints, fingerprints)
        similarity_stats = calculate_similarity_statistics(sim_mat)
        similarity_stats_prefixed = {f"{key}_{desc}": value for key, value in similarity_stats.items()}

        # Update chemdiv_df with similarity statistics for current descriptor
        chemdiv_df = chemdiv_df.assign(**similarity_stats_prefixed)

        # Generate networks and calculate metrics for each threshold
        networks = generate_networks_for_thresholds(sim_mat, df_features["cid"].tolist(), similarity_thresholds)
        network_metrics_ls = []
        for threshold in similarity_thresholds:
            network_metrics = calculate_network_metrics(networks[threshold], name=dataset_name)
            network_metrics['Threshold'] = threshold
            network_metrics_ls.append(network_metrics)
        network_metrics['desc_type'] = desc
        
        # Collect network metrics
        df_network = pd.DataFrame(network_metrics_ls)
        network_analysis_ls.append(df_network)

    # Store chemdiv_df with current dataset's metrics in chemdiv_analysis_ls
    chemdiv_analysis_ls.append(chemdiv_df)

# After the loop, consolidate chemdiv_analysis_ls into a single DataFrame if desired
chemdiv_final_df = pd.concat(chemdiv_analysis_ls, ignore_index=True)


In [149]:
df_features['smi'].parallel_apply(lambda x: Chem.MolToSmiles(MurckoScaffold.GetScaffoldForMol(Chem.MolFromSmiles(x))))

0       O=C(C=Cc1ccccc1-n1cnnn1)NC(CC(=O)N1CCNCC1)c1nc...
1                                    c1ccc(COc2ccccc2)cc1
2                   O=C(CCCc1cccc[nH+]1)NCC(=O)NCc1ccccc1
3         O=C(Nc1ccccc1C(=O)Nc1ccccn1)c1cc(CNC2=NCCS2)cs1
4             O=C(NCB1OC2CC3CC(C3)C2O1)c1ccc(-c2ccccc2)o1
                              ...                        
4015    O=C(CNS(=O)(=O)Cc1ccccc1)NCC(=O)NC(CC1CCCCC1)C...
4016                      O=C(c1ccccc1)n1cnc(-c2cccnc2)n1
4017                      c1ccc(NCc2cccc(OC3CCCCC3)c2)cc1
4018    O=C(C(Cc1nc2ccccc2s1)NS(=O)(=O)c1cccc2c1NCCC2)...
4019      O=C(NCc1ccccc1)C1CCCN1C(=O)CC(c1ccccc1)c1ccccc1
Name: smi, Length: 4020, dtype: object

In [ ]:
df_features['smi'].parallel_apply(Chem.MolToSmiles(MurckoScaffold.GetScaffoldForMol(Chem.MolFromSmiles)))

In [154]:
descriptors

array([[ 4.00000e+00,  1.00000e+01,  2.94110e+00,  6.63526e+02,
         1.78020e+02,  8.00000e+00],
       [ 2.00000e+00,  4.00000e+00,  3.62092e+00,  3.94397e+02,
         9.52200e+01,  8.00000e+00],
       [ 5.00000e+00,  6.00000e+00, -1.59900e-01,  4.63922e+02,
         1.60490e+02,  1.20000e+01],
       ...,
       [ 5.00000e+00,  6.00000e+00,  3.28947e+00,  4.27501e+02,
         1.37890e+02,  9.00000e+00],
       [ 4.00000e+00,  8.00000e+00,  2.86260e+00,  6.48877e+02,
         1.49600e+02,  1.10000e+01],
       [ 6.00000e+00,  8.00000e+00,  2.77387e+00,  5.95637e+02,
         1.70230e+02,  1.20000e+01]])

In [133]:
df_network = pd.concat(network_analysis_ls)

In [134]:
df_network

,Network,Density,Degree Centrality Std Dev,Assortativity Coefficient,Network Entropy,Threshold
0,CHEMBL234,0.379930,0.220073,0.261169,7.224744,0.5
1,CHEMBL234,0.157564,0.133576,0.340882,6.808258,0.6
2,CHEMBL234,0.041935,0.049251,0.419548,5.699516,0.7
3,CHEMBL234,0.006497,0.008271,0.518956,4.052387,0.8
4,CHEMBL234,0.000729,0.001075,0.671226,2.079250,0.9
0,CHEMBL227,0.494308,0.259642,0.288846,5.612461,0.5
1,CHEMBL227,0.291515,0.212914,0.348237,5.598981,0.6
2,CHEMBL227,0.117446,0.115009,0.392371,5.103360,0.7
3,CHEMBL227,0.030218,0.034529,0.588772,3.984565,0.8
4,CHEMBL227,0.005691,0.007972,0.722940,2.538729,0.9


In [75]:
fingerprints.shape

(4020, 1024)

In [78]:
sim_mat.shape

(4020, 4020)

In [110]:
network_metrics

{'Network': b'CHEMBL204',
 'Density': 0.313047848589845,
 'Degree Centrality Std Dev': 0.20309078379706985}

# Read optimization results

## HDF5 File Structure (ambient_dist_and_PCA_results.h5)

This HDF5 file contains datasets related to high-dimensional data and the results of Principal Component Analysis (PCA) performed on this data.

### Datasets
1. **X_HD**: 
   - A high-dimensional dataset with a shape of **(4020, 1024)**.
   - It contains 4020 samples, each with 1024 features.
   - This is the original high-dimensional feature representation of the data.

2. **X_PCA**: 
   - A PCA-transformed dataset with a shape of **(4020, 2)**.
   - It contains the same 4020 samples, but each sample has been reduced to 2 principal components using PCA.
   - This reduced dataset is typically used for visualization or as input for further machine learning analysis.

The file likely stores high-dimensional data and the results of PCA to enable comparison between the original and PCA-reduced data.


## HDF5 File Structure (mfp_r2_1024.h5)

This HDF5 file contains datasets and groups related to various dimensionality reduction techniques applied to the Morgan Fingerprint (radius 2, 1024 bits).

### Datasets and Groups:

1. **GTM_coordinates**:
   - Coordinates of the data after applying Generative Topographic Mapping (GTM).
   - These are likely 2D or 3D coordinates for visualization or analysis.

2. **GTM_metrics**:
   - Performance or quality metrics related to the GTM method.
   - Could contain measures like reconstruction error, stress, or variance explained.

3. **PCA_coordinates**:
   - Coordinates of the data after applying Principal Component Analysis (PCA).
   - This dataset contains the principal component scores, often used to visualize the data in reduced dimensions.

4. **PCA_metrics**:
   - Metrics associated with the PCA method, likely showing the explained variance or other statistics for each principal component.

5. **UMAP_coordinates**:
   - Coordinates after applying Uniform Manifold Approximation and Projection (UMAP).
   - These coordinates represent the data in a lower-dimensional space, typically for clustering or visualization.

6. **UMAP_metrics**:
   - Performance metrics for the UMAP method, possibly including measures like neighborhood preservation.

7. **dataframe**:
   - A tabular dataset, possibly containing metadata or other information related to the features or molecules analyzed in the file.

8. **mfp_r2_1024**:
   - The original Morgan Fingerprint dataset (radius 2, 1024 bits).
   - It encodes molecular features in a binary vector format, used for cheminformatics and molecular modeling.

9. **t-SNE_coordinates**:
   - Coordinates after applying t-Distributed Stochastic Neighbor Embedding (t-SNE).
   - These coordinates are typically used for visualizing high-dimensional data in 2D or 3D.

10. **t-SNE_metrics**:
   - Metrics related to the t-SNE method, possibly including perplexity, KL divergence, or neighborhood preservation.

This file organizes molecular data and its corresponding projections in various dimensionality reduction methods, enabling analysis and comparison of the techniques.


In [10]:
file_path = '../results/in_sample_eucl/CHEMBL204/mfp_r2_1024/mfp_r2_1024.h5'
descriptor_set = 'mfp_r2_1024'
methods_to_extract = ['PCA', 't-SNE', 'UMAP', 'GTM']
df, fp_array, results = read_optimization_results(file_path, feature_name=descriptor_set, method_names=methods_to_extract)

In [11]:
df.head()

,dataset,smi
0,CHEMBL204,CN1CCN(CC1)C(=O)CC(NC(=O)C=Cc1cc(Cl)ccc1-n1cnn...
1,CHEMBL204,Cc1cc(OCCC=NN=C(N)N)cc(OCc2ccccc2C(F)(F)F)c1
2,CHEMBL204,NCc1ccc(Cl)cc1CNC(=O)CNC(=O)C(CCc1cccc[n+]1[O-...
3,CHEMBL204,COc1cc(Cl)cc(C(=O)Nc2ccc(Cl)cn2)c1NC(=O)c1scc(...
4,CHEMBL204,CC1(C)C2CC1C1(C)OB(OC1C2)C(CCCN=C(N)N)NC(=O)c1...


In [13]:
fp_array.shape

(4020, 1024)

In [15]:
results

{'PCA': {'metrics': {'AUC': (np.float64(0.5479564052076352),
    np.float64(0.0006813930286349295)),
   'LCMC': (array([ 2.42665066e-02,  4.45996799e-02,  5.63995198e-02, ...,
           -6.41025333e-07, -1.60192179e-07,  0.00000000e+00]),
    array([0.00475908, 0.00256645, 0.00131993, ..., 0.        , 0.        ,
           0.        ])),
   'QNN': (array([0.02466667, 0.0454    , 0.0576    , ..., 0.99919904, 0.99959968,
           1.        ]),
    array([0.00475908, 0.00256645, 0.00131993, ..., 0.        , 0.        ,
           0.        ])),
   'Qglobal': (np.float64(0.5526558292306168),
    np.float64(0.0010322391467407711)),
   'Qlocal': (np.float64(0.10001809539257533),
    np.float64(0.002001014100452946)),
   'cont_ls': (array([0.93135454, 0.91039158, 0.87825185, 0.831341  , 0.74923995]),
    array([0.00316594, 0.00353948, 0.00313003, 0.00332986, 0.00291319])),
   'kmax': (np.float64(26.666666666666668), np.float64(2.0548046676563256)),
   'nn_overlap': array([ 3.49502488,  6.